## Trabajo Fin de Máster <br/> Diseño de una arquitectura multimodal para descripción textual de pares imagen-audio

## Script 3. Obtención de imágenes a partir de la base de datos

En este notebook, tomamos los vídeos que forman parte de la base de datos y extraemos los fotogramas que los componen. Para ello, hacemos uso de la biblioteca OpenCV.

### Paso 1. Cambio de directorio e importación de paquetes

In [3]:
import os
import cv2
import shutil
import pandas as pd

In [4]:
os.chdir("..")
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/tfm-cpu/code/Users/jose.puche/Scripts'

### Paso 2. Extracción de imágenes
Dentro de la carpeta correspondiente a cada participante, tenemos dos archivos: 'video_sq.mp4' y 'audio.ogg'. Tomamos el primero para convertirlo en imágenes con la ayuda de `cv2`.

Lo que hacemos es tomar el vídeo (de imagen cuadrada) y guardar cada uno de los frames como una imagen independiente de las demás. Se implementa una lógica para seleccionar ciertas imágenes con el objetivo de reducir la redundancia del dataset conseguido.

In [6]:
def ImageExtraction(db_path, new_db_path, output_dim=None):

    participants = os.listdir(db_path)

    rel_paths_train = []
    classID_train = []
    rel_paths_test = []
    classID_test = []
    for person in participants:

        if '.' in person:
            continue

        video = cv2.VideoCapture(f"{db_path}/{person}/video_sq.mp4")
        folder_path = f"{new_db_path}/image/{person}"

        number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        frames_skip_train = round(number_of_frames/100)
        frames_skip_test = round(number_of_frames/100) * 3 + 1

        try:
            # Creamos la carpeta donde guardaremos las imágenes, en caso de que no exista
            if os.path.exists(folder_path):
                shutil.rmtree(folder_path)
            os.makedirs(folder_path)
        except OSError:
            print ('Error: Creating directory of data')

        # Iniciamos la cuenta de fotogramas extraídos
        currentframe = 0
        train_frames = 0
        test_frames = 0

        while(True):

            # Leemos el fotograma. El método read() devuelve una tupla con dos valores:
            # ret. Booleano que muestra si la lectura fue exitosa (True/False)
            # frame. Numpy array (x,y,3) que contiene la imagen leída.
            ret, frame = video.read()

            if ret:
                # Crear imágenes mientras quede vídeo (ret=True)
                name = folder_path + '/frame{:0>5}.jpg'.format(currentframe)

                if currentframe % frames_skip_train == 0:
                    # Escritura del fotograma extraído
                    resized_frame = cv2.resize(frame, output_dim) if output_dim else frame
                    cv2.imwrite(name, resized_frame)
                    classID_train.append(person)
                    rel_paths_train.append(name)
                    train_frames += 1
                elif currentframe % frames_skip_test == 0:
                    # Escritura del fotograma extraído
                    resized_frame = cv2.resize(frame, output_dim) if output_dim else frame
                    cv2.imwrite(name, resized_frame)
                    classID_test.append(person)
                    rel_paths_test.append(name)
                    test_frames +=1

                # Guardamos las dimensiones del primer frame para imprimirlo posteriormente.
                if currentframe == 0:
                    dims = resized_frame.shape

                # Incrementamos la cuenta de fotogramas
                currentframe += 1
            else:
                # Si ya no queda vídeo, salimos del bucle.
                break

        # Imprimimos información sobre el participante empleado.
        print(f"{person}: ({train_frames}+{test_frames}))/{currentframe} frames of {str(dims)}.")

    return pd.DataFrame({ 'image_path': rel_paths_train, 'classID': classID_train }), \
           pd.DataFrame({ 'image_path': rel_paths_test, 'classID': classID_test }),
           

train_df, test_df = ImageExtraction("../Database", "../Final_Database")
display(train_df)

Alba Azorin Zafrilla: (113+22))/339 frames of (480, 480, 3).
Alfonso Girona Palao: (98+28))/780 frames of (480, 480, 3).
Alfonso Vidal Lopez: (106+24))/424 frames of (480, 480, 3).
Ana Azorin Puche: (106+29))/739 frames of (480, 480, 3).
Ana Puche Palao: (98+29))/1069 frames of (480, 480, 3).
Angela Espinosa Martinez: (94+21))/373 frames of (480, 480, 3).
Clara Hidalgo Lopez: (102+27))/612 frames of (480, 480, 3).
Cristina Carpena Ortiz: (103+28))/720 frames of (480, 480, 3).
David Azorin Soriano: (125+0))/125 frames of (480, 480, 3).
Diego Molina Puche: (112+22))/335 frames of (432, 432, 3).
Eva Jimenez Mariscal: (109+22))/325 frames of (480, 480, 3).
Francisco Jose Maldonado Montiel: (98+24))/486 frames of (480, 480, 3).
Genesis Reyes Arteaga: (91+21))/363 frames of (480, 480, 3).
Irene Gutierrez Perez: (105+28))/630 frames of (480, 480, 3).
Irene Molina Puche: (105+21))/315 frames of (480, 480, 3).
Irene Ponte Ibanez: (109+28))/545 frames of (480, 480, 3).
Iria Alonso Alves: (107+28

,image_path,classID
0,../Final_Database/image/Alba Azorin Zafrilla/f...,Alba Azorin Zafrilla
1,../Final_Database/image/Alba Azorin Zafrilla/f...,Alba Azorin Zafrilla
2,../Final_Database/image/Alba Azorin Zafrilla/f...,Alba Azorin Zafrilla
3,../Final_Database/image/Alba Azorin Zafrilla/f...,Alba Azorin Zafrilla
4,../Final_Database/image/Alba Azorin Zafrilla/f...,Alba Azorin Zafrilla
...,...,...
4132,../Final_Database/image/Sergio Castano Azorin/...,Sergio Castano Azorin
4133,../Final_Database/image/Sergio Castano Azorin/...,Sergio Castano Azorin
4134,../Final_Database/image/Sergio Castano Azorin/...,Sergio Castano Azorin
4135,../Final_Database/image/Sergio Castano Azorin/...,Sergio Castano Azorin


In [7]:
train_df.to_csv('../Final_Database/image/imagesDB_train.csv', index=False)
test_df.to_csv('../Final_Database/image/imagesDB_test.csv', index=False)